In [ ]:
!pip install torch==1.12.1  torchtext==0.13.1 torchdata==0.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.3 MB/s 
     |████████████████████████████████| 140 kB 46.9 MB/s 
     |████████████████████████████████| 127 kB 57.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Entrega3')

In [ ]:
f=open("/content/drive/My Drive/Entrega3/spa_eng/eng-spa.txt","r")
for line in f:
    line.split("\tCC-BY")[0]

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import random
from lang_processing import *
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
#     lines = open('spa_eng/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
    lines = open('/content/drive/My Drive/Entrega3/spa_eng/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


    
    
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

    


In [ ]:
input_lang, output_lang, pairs = prepareData('eng', 'spa', True)

Reading lines...
Read 139013 sentence pairs
Trimmed to 8506 sentence pairs
Counting words...
Counted words:
spa 4089
eng 2801


In [ ]:
MAX_LENGTH = 10

In [ ]:
print(len(pairs))

8506


In [ ]:
# rnd_idx = 
print(pairs[11:20])

[['estoy viejo .', 'i am old .'], ['he vuelto .', 'i m back .'], ['estoy de vuelta .', 'i m back .'], [' he vuelto !', 'i m back .'], ['soy calvo .', 'i m bald .'], ['estoy calvo .', 'i m bald .'], ['estoy calmado .', 'i m calm .'], ['estoy tranquilo .', 'i m cool .'], ['soy sordo .', 'i m deaf .']]


In [ ]:
input_lang.word2index

{'tengo': 2,
 'diecinueve': 3,
 '.': 4,
 'estoy': 5,
 'levantado': 6,
 'soy': 7,
 'tom': 8,
 'gordo': 9,
 'gorda': 10,
 'en': 11,
 'forma': 12,
 'afectado': 13,
 'loco': 14,
 'viejo': 15,
 'timido': 16,
 'mojada': 17,
 'he': 18,
 'vuelto': 19,
 'de': 20,
 'vuelta': 21,
 '': 22,
 '!': 23,
 'calvo': 24,
 'calmado': 25,
 'tranquilo': 26,
 'sordo': 27,
 'terminado': 28,
 'acabado': 29,
 'lo': 30,
 'dejo': 31,
 'bastante': 32,
 'facilon': 33,
 'justo': 34,
 'perfectamente': 35,
 'libre': 36,
 'yo': 37,
 'lleno': 38,
 'llena': 39,
 'ya': 40,
 'me': 41,
 'llene': 42,
 'aqui': 43,
 'casa': 44,
 'herido': 45,
 'llego': 46,
 'tarde': 47,
 'vago': 48,
 'perdida': 49,
 'malo': 50,
 'toca': 51,
 'a': 52,
 'mi': 53,
 'pobre': 54,
 'rica': 55,
 'rico': 56,
 'salvo': 57,
 'enferma': 58,
 'flaca': 59,
 'delgado': 60,
 'limpio': 61,
 'fea': 62,
 'calentito': 63,
 'debil': 64,
 'listo': 65,
 'estamos': 66,
 'bien': 67,
 'el': 68,
 'es': 69,
 'anciano': 70,
 'dj': 71,
 'veloz': 72,
 'bueno': 73,
 'saludab

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        output = self.embedding(x).view(1, 1, -1)
#         output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        # print(output.shape)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class AttnDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, max_length=MAX_LENGTH):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
#         self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
#         self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x, hidden, encoder_outputs):
        embedded = self.embedding(x).view(1, 1, -1)
#         embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5


def train_once(input_tensor, target_tensor, encoder, decoder,
          encoder_optimizer, decoder_optimizer, criterion, 
          max_length=MAX_LENGTH, attention=True):
    encoder_hidden = encoder.init_hidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):

            if attention:
              decoder_output, decoder_hidden, decoder_attention = decoder(
                  decoder_input, decoder_hidden, encoder_outputs)
            else:
              decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)


            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):

          if attention:
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
          else:
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)


          topv, topi = decoder_output.topk(1)
          decoder_input = topi.squeeze().detach()  # detach from history as input

          loss += criterion(decoder_output, target_tensor[di])
          if decoder_input.item() == EOS_token:
              break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
random.choice(pairs)

['eres solo un chico .', 'you re just a boy .']

In [ ]:
def train(encoder, decoder, n_iters, print_every=1000, 
               plot_every=100, learning_rate=0.01, attention=True):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train_once(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, 
                     criterion, attention=attention)
        
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH, attention=True):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.init_hidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
          if attention:
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
          else:
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)


          if topi.item() == EOS_token:
              decoded_words.append('<EOS>')
              break
          else:
              decoded_words.append(output_lang.index2word[topi.item()])

          decoder_input = topi.squeeze().detach()

        if attention:
          return decoded_words, decoder_attentions[:di + 1]
        else:
          return decoded_words

In [ ]:
def evaluateRandomly(encoder, decoder, n=10, attention=True):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        if attention:
          output_words, attentions = evaluate(encoder, decoder, pair[0], attention=attention)
        else:
          output_words = evaluate(encoder, decoder, pair[0], attention=False)

        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = Encoder(input_lang.n_words, hidden_size).to(device)
decoder1 = AttnDecoder(hidden_size, output_lang.n_words).to(device)
#decoder1 = Decoder(hidden_size, output_lang.n_words).to(device)
train(encoder1, decoder1, 50000, print_every=5000, attention=True)

8m 7s (- 73m 4s) (5000 10%) 3.0589
16m 2s (- 64m 8s) (10000 20%) 2.3869
24m 1s (- 56m 2s) (15000 30%) 2.0413
31m 56s (- 47m 54s) (20000 40%) 1.7709
40m 11s (- 40m 11s) (25000 50%) 1.5215
48m 29s (- 32m 19s) (30000 60%) 1.3246
56m 30s (- 24m 13s) (35000 70%) 1.1836
64m 26s (- 16m 6s) (40000 80%) 1.0492
72m 39s (- 8m 4s) (45000 90%) 0.9329
80m 39s (- 0m 0s) (50000 100%) 0.8340


In [ ]:
evaluateRandomly(encoder1, decoder1, attention=True)

> estoy satisfecho con todo .
= i m satisfied with everything .
< i m satisfied with everything . <EOS>

> el habla chino fluido .
= he is fluent in chinese .
< he is just in chinese . <EOS>

> a el le dan miedo las serpientes .
= he is afraid of snakes .
< he s scared of snakes . <EOS>

> estamos acostumbrados a llevar zapatos .
= we re accustomed to wearing shoes .
< we re accustomed to wearing shoes . <EOS>

> estoy contento de verte .
= i m happy to see you .
< i am happy to see you . <EOS>

> no tengo palabras .
= i m at a loss for words .
< i m not a member . <EOS>

> el no es un tipo muy meticuloso .
= he s not a very meticulous guy .
< he s not a very intelligent very . <EOS>

> voy en avion .
= i m going by plane .
< i am going to plane . <EOS>

> acabe mi tarea .
= i m done with my homework .
< i m giving my homework . <EOS>

> estoy feliz de verte aqui .
= i am happy to see you here .
< i m happy to see you . <EOS>

